In [1]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk import word_tokenize
from collections import defaultdict
from nltk.stem import WordNetLemmatizer
import string
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [3]:
def delete(train):
    del train['location']
    del train['keyword']
    return train

In [4]:
delete(train)
delete(test)
y_train=train.iloc[:,-1]
x_train=train.iloc[:, -1]

In [5]:
lemm = WordNetLemmatizer()

In [6]:
#importing a comprehensive list of stopwords from GITHUB, make sure to cite when writing final report
gist_file = open("gist_stopwords.txt", "r")
try:
    content = gist_file.read()
    stopwords = content.split(",")
finally:
    gist_file.close()

In [7]:
def stopwordremoval(text):
    return " ".join([w for w in str(text).split() if w not in stopwords])

In [8]:
#removing puncuation from text
import string 
englishpunc = string.punctuation
punctuations_list = englishpunc
def punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)
train['text']= train['text'].apply(lambda x: punctuations(x))
train['text'].head()

0    Our Deeds are the Reason of this earthquake Ma...
1                Forest fire near La Ronge Sask Canada
2    All residents asked to shelter in place are be...
3    13000 people receive wildfires evacuation orde...
4    Just got sent this photo from Ruby Alaska as s...
Name: text, dtype: object

In [9]:
def cleaning_URLs(data):
    return re.sub('((www.[^s]+)|(https?://[^s]+))',' ',data)

In [10]:
array = []
def arraycreator(data):
    data = data.lower()
    array.append(data)

In [11]:
def processdata(train):
    train['text'] = train['text'].apply(lambda x: cleaning_URLs(x))
    train['text'] = train['text'].apply(lambda x: stopwordremoval(x))
    train['text']= train['text'].apply(lambda x: punctuations(x))
    train['text'].apply(lambda x: arraycreator(x))
    return array

In [12]:
train = processdata(train)
test = processdata(test)

In [13]:
cv=TfidfVectorizer(max_features=8000)
y_train.shape

(7613,)

In [14]:
x_train_vector=cv.fit_transform(train).toarray()
x_test_vector=cv.transform(test).toarray()
x_train_vector = x_train_vector[:7613]
x_train_vector.shape

(7613, 8000)

In [15]:
from sklearn.naive_bayes import MultinomialNB
model=MultinomialNB()
model.fit(x_train_vector,y_train)
print(model.score(x_train_vector,y_train))
y_pred1=model.predict(x_test_vector)

0.8690398003415211


In [16]:
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier(random_state=4,n_estimators=500,warm_start=True,max_depth=6,min_samples_leaf=2,min_samples_split=3)
rfc.fit(x_train_vector,y_train)
print(rfc.score(x_train_vector,y_train))
y_pred2=rfc.predict(x_test_vector)

0.6224878497307238


In [17]:
from xgboost import XGBClassifier
xgb=XGBClassifier()
xgb.fit(x_train_vector,y_train,early_stopping_rounds=5, 
             eval_set=[(x_train_vector,y_train)], 
             verbose=False)
print(xgb.score(x_train_vector,y_train))
y_pred3=xgb.predict(x_test_vector)

/Users/sharanvamsi/anaconda3/lib/python3.11/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


0.8392223827663208


In [18]:
from sklearn.linear_model import LogisticRegression
reg=LogisticRegression()
reg.fit(x_train_vector,y_train)
print(reg.score(x_train_vector,y_train))
y_pred4=reg.predict(x_test_vector)

0.8788913700249573


In [25]:
from sklearn.linear_model import PassiveAggressiveClassifier
passive=PassiveAggressiveClassifier()
passive.fit(x_train_vector,y_train)
print(passive.score(x_train_vector,y_train))
y_pred5=passive.predict(x_test_vector)

0.9808222776829109
